# Initiation

In [ ]:
import boto3
import json
import importlib
import os
import sys
import time
sys.path.append("../")

from dotenv import load_dotenv
from utils.openaiClient import OpenAIClient

# DEV AUTO RELOAD - REMOVE IN PRODUCTION
import textractParser
import textractClient
importlib.reload(textractClient)
importlib.reload(textractParser)

from textractClient import *
from textractParser import *

# Load the .env file (assuming it’s in the same folder)
load_dotenv()
AWS_REGION = os.getenv("AWS_REGION")

boto3.setup_default_session(
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=AWS_REGION
)

openai_client = OpenAIClient(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# CONFIGURATION
catalog_names = ["ms-01-138",
"MS-01-140",
"ms-01-146",
"ms-01-147",
"ms-01-149",
"ms-01-172",
"ms-01-176",
"ms-01-179",
"ms-01-42",
"MS-01-91",
"ms-01-98",
"MS-02-170",
"ms-02-225",
"ms-02-331",
"ms-02-89",
"ms-02-89"]
# "ms-02-02",

CATALOG_NAME = 'ms-02-02'
CATALOG_FILE = f'{CATALOG_NAME}.pdf'

S3_BUCKET = 'catalogs-for-extraction'
S3_CATALOG_PATH = f's3://{S3_BUCKET}/{CATALOG_FILE}'

TEXTRACT_OUTPUT_JSON_PATH = f"textract_outputs/textract_output_{CATALOG_NAME}.json"

# Run Textract Job

In [ ]:
# job_id = start_job(s3_bucket, document, features=['TABLES'], region=AWS_REGION)
job_id = "bce4b0bc30439528968cc52ca5ae225868056799096afbcc6944bdaef4fe81df"

status = is_job_complete(job_id, region=AWS_REGION)
results_pages = get_job_results(job_id, region=AWS_REGION)
save_results_to_file(results_pages, TEXTRACT_OUTPUT_JSON_PATH)

# Parse textract results into tables

In [ ]:
# LOAD BLOCKS FROM TEXTTRACT
pages =      load_textract_pages(TEXTRACT_OUTPUT_JSON_PATH)
all_blocks = convert_pages_to_blocks(pages)

# CREATE MAPPING OF ALL BLOCKS AND GET ONLY TABLE BLOCKS
id_map, type_map = build_block_maps(all_blocks)
table_blocks = type_map.get("TABLE", [])
print(f"Found {len(table_blocks)} tables in the document")

In [ ]:
# PICK RELEVANT TABLE BLOCK
tindex = 2  
tblock = table_blocks[tindex]

In [ ]:
# GET SPECIAL CELLS - COLUMN_HEADER, TABLE_TITLE, TABLE_FOOTER, TABLE_SECTION_TITLE, TABLE_SUMMARY
special_types =    get_special_cells_texts(tblock, id_map)
header_row_index = get_header_row_count(special_types.get('COLUMN_HEADER'))
print("row header index: ", header_row_index)

# PARSE TABLES FROM BLOCKS INTO TABLUAR DATAFRAME
table_grid =            convert_table_block_to_grid(tblock, id_map, replicate_data=True, header_scan_rows=header_row_index)
ordering_number_index = has_ordering_number_header(table_grid.get('headers'))

if ordering_number_index:
    table_df = convert_grid_result_to_dataframe(table_grid)
    print("Converted table grid to dataframe")

else:
    table_df = None
    print("Table does not contain ordering number header")


# Prep for event

In [ ]:
event_payload = convert_grid_to_catalog_products(table_grid, tblock, id_map, tindex)

In [ ]:
event_payload